# 0 - IMPORTS

In [55]:
import tensorflow as tf
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

# 1 - LOAD DATASET

In [3]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

# 2 - DATA TRANSFORMATION

In [11]:
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)

In [23]:
# reshape target variable to matrix format "N lines x 1 column" 
y = y.reshape(-1, 1)

# apply one hot encoder to target variable
ct = ColumnTransformer(
    transformers=[
        ("Country",        # Just a name
         OneHotEncoder(), # The transformer class
         [0]            # The column(s) to be applied on.
         )
    ], remainder='passthrough'
)

y = ct.fit_transform(y)
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

# 3 - CREATE TRAIN & TEST DATASET

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [27]:
X_train.shape

(105, 4)

In [28]:
X_test.shape

(45, 4)

# 4 - CREATE TENSORFLOW MODEL

![x_and_y](img/neural_network.png)

## 4.1 Create input layer

In [30]:
input_layer = X.shape[1]
print('input_layer contains {} neurons.'.format(input_layer))

input_layer contains 4 neurons.


## 4.2 Create hidden layer

In [31]:
hidden_layer = int(np.ceil((X.shape[1] + y.shape[1]) / 2))
print('hidden_layer contains {} neurons.'.format(hidden_layer))

hidden_layer contains 4 neurons.


## 4.3 Create output layer

In [32]:
output_layer = y.shape[1]
print('hidden_layer contains {} neurons.'.format(output_layer))

hidden_layer contains 3 neurons.


## 4.4 Create Weights Tensor Variables

In [39]:
W = {'hidden': tf.Variable(tf.random_normal([input_layer, hidden_layer])),
     'output': tf.Variable(tf.random_normal([hidden_layer, output_layer]))}

## 4.5 Create Bias Tensor Variables 

In [40]:
b = {'hidden': tf.Variable(tf.random_normal([hidden_layer])),
     'output': tf.Variable(tf.random_normal([output_layer]))}

## 4.6 Create PlaceHolders Tensor Variables 

In [36]:
xph = tf.placeholder('float', [None, input_layer])
yph = tf.placeholder('float', [None, output_layer])

## 4.7 Create Tensorflow Model

In [37]:
# define multilayer perceptron function
def mlp(x, W, bias):
    input_layer_to_hidden_layer = tf.add(tf.matmul(x, W['hidden']), bias['hidden'])
    hidden_layer_activation     = tf.nn.relu(input_layer_to_hidden_layer)
    output_layer                = tf.add(tf.matmul(hidden_layer_activation, W['output']), b['output'])
    return output_layer

In [43]:
model = mlp(xph, W, b)

## 4.8 Create Model Optimizer

In [45]:
model_error     = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = yph))
model_optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(model_error)

# 5 - TRAINING MODEL

## 5.1 - Define Batch Structure

In [46]:
batch_size  = 8 # 8 records to train and optmize model
batch_cycle = int(len(X_train) / batch_size) # number of cicles to train all records
print( 'We have {} cicles with {} records each.'.format(batch_cycle, batch_size) )

We have 13 cicles with 8 records each.


In [33]:
# create array of batch with predict variables
X_batches = np.array_split(X_train, batch_cycle)
X_batches

[array([[-0.29484182, -0.8191665 ,  0.25096731,  0.13322594],
        [-0.41600969, -1.51337539,  0.02350449, -0.12972   ],
        [ 1.2803405 ,  0.33784833,  1.10395287,  1.44795564],
        [ 1.03800476,  0.10644536,  0.36469871,  0.26469891],
        [ 1.03800476,  0.10644536,  0.53529583,  0.39617188],
        [-0.7795133 ,  0.80065426, -1.3412724 , -1.31297673],
        [-1.02184904, -1.74477836, -0.26082403, -0.26119297],
        [-1.50652052,  1.26346019, -1.56873522, -1.31297673],
        [-0.05250608, -0.8191665 ,  0.76275864,  0.92206376]]),
 array([[ 0.4321654 , -0.58776353,  0.59216153,  0.79059079],
        [-0.90068117,  1.72626612, -1.2844067 , -1.18150376],
        [ 1.15917263,  0.33784833,  1.21768427,  1.44795564],
        [ 0.31099753, -1.05056946,  1.04708716,  0.26469891],
        [-0.7795133 ,  1.03205722, -1.2844067 , -1.31297673],
        [-0.05250608, -1.05056946,  0.1372359 ,  0.00175297],
        [-0.17367395, -0.58776353,  0.42156442,  0.13322594],
      

## 5.2 - Model Training

In [47]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(3000):
        mean_error = 0.0
        batch_cycle = int(len(X_train) / batch_size)
        X_batches = np.array_split(X_train, batch_cycle)
        y_batches = np.array_split(y_train, batch_cycle)
        for i in range(batch_cycle):
            X_batch, y_batch = X_batches[i], y_batches[i]
            _, cost = sess.run([model_optimizer, model_error], feed_dict = {xph: X_batch, yph: y_batch})
            mean_error += cost / batch_cycle
        if epoch % 500 == 0:
            print('Epoch: ' + str((epoch + 1)) + ' error: ' + str(mean_error))
    W_final, b_final = sess.run([W, b])

Epoch: 1 error: 1.2724829407838676
Epoch: 501 error: 0.4546482127446395
Epoch: 1001 error: 0.2822469318142304
Epoch: 1501 error: 0.18963478047114155
Epoch: 2001 error: 0.12761626994380584
Epoch: 2501 error: 0.08571971365465567


## 5.3 - Final Weights

In [52]:
W_final

{'oculta': array([[-0.5557124 , -1.8534161 ,  0.15347278,  0.9356429 ],
        [-0.30108556,  1.1704956 ,  1.5173255 ,  0.1725314 ],
        [ 3.2615788 , -1.5167435 , -0.01237538,  0.8732984 ],
        [ 1.229192  , -1.5423791 , -0.12267964,  0.34158868]],
       dtype=float32), 'saida': array([[-2.496203 , -1.3808637,  1.4181937],
        [ 2.0235126, -1.6290889, -1.4720006],
        [-0.2312544,  0.5091364, -1.0839738],
        [-0.879805 , -2.9005344,  1.1711245]], dtype=float32)}

In [53]:
b_final

{'oculta': array([-1.5457541, -1.1545649, -2.7522078, -1.791996 ], dtype=float32),
 'saida': array([-1.67663  ,  2.3390503, -0.8624562], dtype=float32)}

# 6 - PREDICTIONS

In [49]:
# predictions
model_predictions = mlp(xph, W_final, b_final)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(model_predictions, feed_dict = {xph: X_test})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

# 6.1 - Output Layer Results

In [50]:
r1

array([[-12.274862  ,  -3.9607048 ,  -0.35668147],
       [  9.162098  ,  -7.360831  , -20.307787  ],
       [ -5.188447  ,  -0.6265671 ,  -3.7338743 ],
       [  9.909658  ,  -7.9546695 , -21.151772  ],
       [ -7.3711853 ,   0.6805234 ,  -3.5102544 ],
       [  8.538536  ,  -6.858585  , -18.858908  ],
       [ -4.30509   ,  -0.36674845,  -2.7716184 ],
       [-12.064908  ,  -3.236835  ,   0.2012552 ],
       [-10.000791  ,  -1.8623134 ,  -0.78861034],
       [ -9.916374  ,  -3.017314  ,  -0.7285273 ],
       [ -4.2571797 ,  -0.299922  ,  -2.7034006 ],
       [ -8.32328   ,  -1.5482289 ,  -4.7426744 ],
       [  8.253269  ,  -6.778747  , -18.807182  ],
       [ -4.031439  ,   2.0901437 ,  -5.8207054 ],
       [ -3.44696   ,   0.17411083,  -4.479288  ],
       [ -4.6724176 ,   0.5559977 ,  -4.007749  ],
       [-11.664723  ,  -1.5805022 ,  -1.320839  ],
       [  6.265619  ,  -5.58852   , -19.573614  ],
       [-12.656747  ,  -4.4620714 ,   1.280522  ],
       [ -7.905629  ,   1.34314

# 6.2 - Softmax Output Layer

In [51]:
r2

array([[6.49136791e-06, 2.64928527e-02, 9.73500609e-01],
       [9.99999881e-01, 6.67088429e-08, 1.58997964e-13],
       [9.89648141e-03, 9.47720289e-01, 4.23832089e-02],
       [1.00000000e+00, 1.74429822e-08, 3.23737009e-14],
       [3.13709519e-04, 9.84782159e-01, 1.49041945e-02],
       [9.99999762e-01, 2.05643531e-07, 1.26311017e-12],
       [1.75538287e-02, 9.01097655e-01, 8.13485980e-02],
       [4.56182306e-06, 3.11258789e-02, 9.68869507e-01],
       [7.43875207e-05, 2.54680574e-01, 7.45245039e-01],
       [9.28511872e-05, 9.20473710e-02, 9.07859802e-01],
       [1.72286108e-02, 9.01291966e-01, 8.14794376e-02],
       [1.09574653e-03, 9.59572077e-01, 3.93321328e-02],
       [9.99999762e-01, 2.96263380e-07, 1.76927407e-12],
       [2.18937127e-03, 9.97444868e-01, 3.65805958e-04],
       [2.58172788e-02, 9.64987218e-01, 9.19551495e-03],
       [5.27869072e-03, 9.84460294e-01, 1.02609629e-02],
       [1.81721061e-05, 4.35438603e-01, 5.64543188e-01],
       [9.99992847e-01, 7.10901

# 6.3 - Argmax Output Layer

In [52]:
r3

array([2, 0, 1, 0, 1, 0, 1, 2, 2, 2, 1, 1, 0, 1, 1, 1, 2, 0, 2, 1, 2, 2,
       0, 2, 0, 0, 0, 0, 1, 2, 0, 2, 2, 2, 1, 0, 1, 1, 0, 0, 2, 2, 1, 1,
       0], dtype=int64)

# 6.4 - Accuracy Score

In [57]:
y_test2 = np.argmax(y_test, 1)
y_test2

array([2, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1, 2, 2,
       0, 2, 0, 0, 0, 0, 1, 2, 0, 2, 2, 2, 1, 0, 2, 1, 0, 0, 2, 2, 1, 1,
       0], dtype=int64)

In [59]:
model_accuracy_score = accuracy_score(y_test2, r3)
model_accuracy_score

0.9111111111111111